In [ ]:
import os

_ROOT_DIR = os.path.join('..', '..', '..')
_EXP_DIR = os.path.join(_ROOT_DIR, 'experiments')
_DATASET_DIR = os.path.join(_EXP_DIR, 'datasets')
_RESULTS_DIR = os.path.join(_EXP_DIR, 'results')

import sys
sys.path.append('..')
import plotconfig

sys.path.append(os.path.join(_ROOT_DIR, 'tools'))
import file_tools
import face_tools
import saving_tools

In [ ]:
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [ ]:
def make_values_unique(arr, epsilon=1e-6):
    # Create a copy of the input array to avoid modifying the original
    unique_arr = arr.astype(float)  # Convert to float to allow adding small values

    # Find the indices of duplicate values
    _, indices = np.unique(unique_arr, return_inverse=True)
    duplicates = np.where(np.bincount(indices) > 1)[0]

    # Add random values to duplicate elements
    for duplicate in duplicates:
        mask = indices == duplicate
        unique_arr[mask] += np.random.uniform(-epsilon, epsilon, size=mask.sum())

    assert len(np.unique(unique_arr)) == len(unique_arr), "The resulting array does not have all unique elements."
            
    return unique_arr

In [ ]:
# EXAMPLE FOR ONE FILE

In [ ]:
all_result_files = file_tools.list_files(_RESULTS_DIR, '*.json')

In [ ]:
result_filename = all_result_files[2]
result_filename

In [ ]:
results = saving_tools.load_dict_from_json(result_filename)
target_data = np.load(results['target_filename'])
test_data = np.load(results['test_filename'])

raw_scores =  np.array([-np.mean(r['raw_scores']['test_neg_root_mean_squared_error']) for r in results['test_scorings']])
shuffled_scores = np.array([-np.mean(r['shuffled_scores']['test_neg_root_mean_squared_error']) for r in results['test_scorings']])

scores = np.array([np.mean(np.array(r['shuffled_scores']['test_neg_root_mean_squared_error']) / np.array(r['raw_scores']['test_neg_root_mean_squared_error'])) for r in results['test_scorings']])
scores = make_values_unique(scores)
sorted_indexes = np.flip(np.argsort(scores))

target = target_data['target_face']
test_faces = test_data['test_faces']
y_true = test_data['true_distances']

In [ ]:
f = test_faces[sorted_indexes[0], :]
face_tools.img_from_latent(f)

In [ ]:
face_tools.img_from_latent(target)

In [ ]:
#
results_folders = sorted(list(file_tools.list_folders(_RESULTS_DIR)))

method_names = ['LinearRegression', 'Shuffle_LinearRegression', 'DummyScoring_Mean']
test_names = ['random', 'diagonal']
eeg_type = 'EEG_Raw'
file_of_interest = '9234.json'

In [ ]:
random.seed(42)
np.random.seed(42)

for test_name in test_names:
    
    rows, cols = len(results_folders), len(method_names)+1
    fig, ax = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
        
    for row, results_folder in enumerate(results_folders):
        for col, method_name in enumerate(method_names):

            xp_folder = random.choice(sorted(list(file_tools.list_folders(os.path.join(results_folder, eeg_type, method_name)))))

            xp_number = xp_folder.split('/')[-1]

            result_filename = os.path.join(xp_folder, test_name, file_of_interest)
            
            results = saving_tools.load_dict_from_json(result_filename)
            target_data = np.load(results['target_filename'])
            test_data = np.load(results['test_filename'])

            scores = np.array([np.mean(np.array(r['shuffled_scores']['test_neg_root_mean_squared_error']) / np.array(r['raw_scores']['test_neg_root_mean_squared_error'])) for r in results['test_scorings']])
            scores = make_values_unique(scores)
            sorted_indexes = np.flip(np.argsort(scores))

            target_face = target_data['target_face']
            test_faces = test_data['test_faces']
            y_true = test_data['true_distances']
            
            f = test_faces[sorted_indexes[0], :]
            img = face_tools.img_from_latent(f, size=1024)
            
            ax[row, col+1].imshow(img)
            ax[row, col+1].set_title(method_name)
            ax[row, col+1].set_xticks([])
            ax[row, col+1].set_yticks([])

            # print("###")
            print(result_filename)
            # print(scores[sorted_indexes[0]])
            # print(f)

            
        # target face plot now, same for all results_folder
        img = face_tools.img_from_latent(target_face)
        ax[row, 0].imshow(img)
        ax[row, 0].set_title('Target {}'.format(row))
        ax[row, 0].set_xticks([])
        ax[row, 0].set_yticks([])

    plotconfig.save_fig(test_name)

In [ ]:
random.seed(42)

for test_name in test_names:  
        
    for row, results_folder in enumerate(results_folders):
        
        fig, ax = plt.subplots(3, 10, figsize=(4*10, 4*3))
        
        rnd_index = random.randint(0, 9)
        
        for col, method_name in enumerate(method_names):

            xp_folder = sorted(list(file_tools.list_folders(os.path.join(results_folder, eeg_type, method_name))))[rnd_index]

            xp_number = xp_folder.split('/')[-1]

            result_filename = os.path.join(xp_folder, test_name, file_of_interest)
            
            print(result_filename) 
            
            results = saving_tools.load_dict_from_json(result_filename)
            target_data = np.load(results['target_filename'])
            test_data = np.load(results['test_filename'])

            raw_scores =  np.array([-np.mean(r['raw_scores']['test_neg_root_mean_squared_error']) for r in results['test_scorings']])
            shuffled_scores = np.array([-np.mean(r['shuffled_scores']['test_neg_root_mean_squared_error']) for r in results['test_scorings']])
            
            scores = np.array([np.mean(np.array(r['shuffled_scores']['test_neg_root_mean_squared_error']) / np.array(r['raw_scores']['test_neg_root_mean_squared_error'])) for r in results['test_scorings']])
            scores = make_values_unique(scores)
            sorted_indexes = np.flip(np.argsort(scores))

            target_face = target_data['target_face']
            test_faces = test_data['test_faces']
            y_true = test_data['true_distances']
            
            for i in range(10):
            
                f = test_faces[sorted_indexes[i], :]
                img = face_tools.img_from_latent(f, size=1024)

                ax[col, i].imshow(img)
                ax[col, i].set_title('{} | R {} | D {}'.format(method_name, i+1, round(y_true[sorted_indexes[i]], 1)))
                ax[col, i].set_xticks([])
                ax[col, i].set_yticks([])
            
#         # target face plot now, same for all results_folder
#         img = face_tools.img_from_latent(target_face)
#         ax[0].imshow(img)
#         ax[0].set_title('Target {}'.format(row))
#         ax[0].set_xticks([])
#         ax[0].set_yticks([])

        top10_dir = 'top10'
        file_tools.ensure_dir(top10_dir)
        fname = os.path.join(top10_dir, '{}_{}'.format(test_name, row))
        plotconfig.save_fig(fname)

In [ ]:
for test_name in test_names:
    
    rows, cols = len(results_folders), len(method_names)+1
    fig, ax = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
        
    for row, results_folder in enumerate(results_folders):
        for col, method_name in enumerate(method_names):

            xp_folders = sorted(list(file_tools.list_folders(os.path.join(results_folder, eeg_type, method_name))))
            
            top_faces = []
            for xp_folder in xp_folders:
                xp_number = xp_folder.split('/')[-1]

                result_filename = os.path.join(xp_folder, test_name, file_of_interest)

                print(result_filename) 

                results = saving_tools.load_dict_from_json(result_filename)
                target_data = np.load(results['target_filename'])
                test_data = np.load(results['test_filename'])
    
                raw_scores =  np.array([-np.mean(r['raw_scores']['test_neg_root_mean_squared_error']) for r in results['test_scorings']])
                shuffled_scores = np.array([-np.mean(r['shuffled_scores']['test_neg_root_mean_squared_error']) for r in results['test_scorings']])
                
                scores = np.array([np.mean(np.array(r['shuffled_scores']['test_neg_root_mean_squared_error']) / np.array(r['raw_scores']['test_neg_root_mean_squared_error'])) for r in results['test_scorings']])
                scores = make_values_unique(scores)
                sorted_indexes = np.flip(np.argsort(scores))
    
                target_face = target_data['target_face']
                test_faces = test_data['test_faces']
                y_true = test_data['true_distances']
                    
                f = test_faces[sorted_indexes[0], :]
                top_faces.append(f)
        
            
            img = face_tools.img_from_latent(np.mean(top_faces, axis=0), size=1024)
            
            ax[row, col+1].imshow(img)
            ax[row, col+1].set_title(method_name)
            ax[row, col+1].set_xticks([])
            ax[row, col+1].set_yticks([])
            
        # target face plot now, same for all results_folder
        img = face_tools.img_from_latent(target_face, size=1024)
        ax[row, 0].imshow(img)
        ax[row, 0].set_title('Target {}'.format(row))
        ax[row, 0].set_xticks([])
        ax[row, 0].set_yticks([])
        
    save_dir = 'merged'
    file_tools.ensure_dir(save_dir)
    fname = os.path.join(save_dir, '{}_{}'.format(test_name, row))
    plotconfig.save_fig(fname)

In [ ]:
tmp = test_data['true_distances']

In [ ]:
tmp.max()/2